In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

from sklearn.impute import KNNImputer

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Adding the data

In [ ]:
train_Data = pd.read_csv('../input/titanic/train.csv')
test_Data = pd.read_csv('../input/titanic/test.csv')
combine = [train_Data, test_Data]

train_Data.head()

In [ ]:
train_Data.describe()

In [ ]:
train_Data.isnull().sum()

Drop features we dont want to use

In [ ]:
train_Data = train_Data.drop(['Name','Ticket','Cabin','PassengerId'], axis=1)
test_Data = test_Data.drop(['Name','Ticket','Cabin'], axis=1)
combine = [train_Data, test_Data]

In [ ]:
train_Data['Sex'] = train_Data['Sex'].map({'male':0, 'female':1}).astype(int)
test_Data['Sex'] = test_Data['Sex'].map({'male':0, 'female':1}).astype(int)
combine = [train_Data, test_Data]

train_Data.head()

In [ ]:
train_Data.Embarked.mode()

In [ ]:
train_Data['Embarked'] = train_Data['Embarked'].fillna('S')
test_Data['Embarked'] = test_Data['Embarked'].fillna('S')


train_Data['Embarked'] = train_Data['Embarked'].map({'C':0, 'Q':1, 'S':2}).astype(int)
test_Data['Embarked'] = test_Data['Embarked'].map({'C':0, 'Q':1, 'S':2}).astype(int)
combine = [train_Data, test_Data]

train_Data.head()

In [ ]:
train_Data['famSize'] = train_Data['SibSp'] + train_Data['Parch'] + 1
test_Data['famSize'] = test_Data['SibSp'] + test_Data['Parch'] + 1

train_Data.head()

Using a KNNImputer to fill missing values of age from Fare and Sibsp features

In [ ]:
knn_imputer = KNNImputer()

imp_Train_Data = knn_imputer.fit_transform(train_Data[['Fare','SibSp','Age']])
imp_Test_Data = knn_imputer.fit_transform(test_Data[['Fare','SibSp','Age']])

train_Data['Age'] = imp_Train_Data[:,2]
test_Data['Age'] = imp_Test_Data[:,2]

Normalizing the Fare feature

In [ ]:
test_Data['Fare'].fillna(value=test_Data.Fare.mean(),inplace=True)
train_Data['Fare'] = train_Data['Fare']/train_Data['Fare'].abs().max()
test_Data['Fare'] = test_Data['Fare']/test_Data['Fare'].abs().max()
train_Data.describe()

In [ ]:
test_Data['Age'].fillna(value=test_Data.Age.mean(),inplace=True)
train_Data['Age'] = train_Data['Age']/train_Data['Age'].abs().max()
test_Data['Age'] = test_Data['Age']/test_Data['Age'].abs().max()
train_Data.describe()

In [ ]:
train_Data[['famSize', 'Survived']].groupby(['famSize'], as_index=False).mean().sort_values(by='famSize', ascending=False)

In [ ]:
combine = [train_Data,test_Data]
for dataset in combine:
    dataset.loc[ dataset['famSize'] <= 1, 'famSize'] = 0
    dataset.loc[(dataset['famSize'] > 1) & (dataset['famSize'] <= 4), 'famSize'] = 1
    dataset.loc[(dataset['famSize'] > 4), 'famSize']   = 2
    dataset['famSize'] = dataset['famSize'].astype(int)

combine_Data = [train_Data, test_Data]

train_Data.head()

In [ ]:
X_train_Data = train_Data.drop("Survived", axis=1)
Y_train_Data = train_Data["Survived"]
X_test_Data  = test_Data.drop("PassengerId", axis=1).copy()
X_train_Data.shape, Y_train_Data.shape, X_test_Data.shape

In [ ]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train_Data, Y_train_Data)
Y_prediction = svc.predict(X_test_Data)
SVM = round(svc.score(X_train_Data, Y_train_Data) * 100, 2)
SVM


This cell finds the best number of neighbors to use for our model

In [ ]:
Hyperparameters = {"n_neighbors": range(1, 50)}
grid_Search = GridSearchCV(KNeighborsClassifier(), Hyperparameters)
grid_Search.fit(X_train_Data, Y_train_Data)

grid_Search.best_params_

In [ ]:
kNN = KNeighborsClassifier(n_neighbors = 14)
kNN.fit(X_train_Data, Y_train_Data)
Y_prediction = kNN.predict(X_test_Data)
KNN = round(kNN.score(X_train_Data, Y_train_Data) * 100, 2)
kNN.score(X_train_Data, Y_train_Data)

In [ ]:
# Decision Tree

decision_Tree = DecisionTreeClassifier()
decision_Tree.fit(X_train_Data, Y_train_Data)
Y_prediction = decision_Tree.predict(X_test_Data)
Decision_Tree= round(decision_Tree.score(X_train_Data, Y_train_Data) * 100, 2)
Decision_Tree

In [ ]:
#parameters = {"n_estimators":[1000,2500, 5000] ,"max_depth": [3,5,7,8,10], "min_samples_split": [5,8,10]}
#gridsearch = GridSearchCV(RandomForestClassifier(), parameters)
#gridsearch.fit(X_train, Y_train)

#gridsearch.best_params_
#{'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 1000}

In [ ]:
# Random Forest

RF = RandomForestClassifier(n_estimators=5000, min_samples_split = 10, max_depth=5)
RF.fit(X_train_Data, Y_train_Data)
Y_pred = RF.predict(X_test_Data)
RF.score(X_train_Data, Y_train_Data)
RANDOMFOREST = round(RF.score(X_train_Data, Y_train_Data) * 100, 2)
RANDOMFOREST

In [ ]:
gbm = xgb.XGBClassifier(learning_rate = .15, max_depth=7,subsample = .7)
gbm.fit(X_train_Data, Y_train_Data)
gmbScore = round(gbm.score(X_train_Data, Y_train_Data) *100, 2)
gmbScore

In [ ]:
# Gaussian Naive Bayes
gaussian = GaussianNB()
gaussian.fit(X_train_Data, Y_train_Data)
Y_pred = gaussian.predict(X_test_Data)
acc_gaussian = round(gaussian.score(X_train_Data, Y_train_Data) * 100, 2)
acc_gaussian

In [ ]:
# test the accuracy of Stochastic Gradient Descent
sgd = SGDClassifier()
sgd.fit(X_train_Data, Y_train_Data)
y_pred = sgd.predict(X_test_Data)
acc_sgd = round(sgd.score(X_train_Data, Y_train_Data) * 100, 2)
acc_sgd

In [ ]:
# test the accuracy of Linear SVC


linear_svc = LinearSVC()
linear_svc.fit(X_train_Data, Y_train_Data)
y_pred = linear_svc.predict(X_test_Data)
acc_linear_svc = round(linear_svc.score(X_train_Data, Y_train_Data) * 100, 2)
acc_linear_svc

In [ ]:
models = pd.DataFrame({
    'Model': [ 'KNN','Random Forest','Decision Tree', 'XGB' , 'GNB','SGD', 'Lin-SVC'],
    'Score': [KNN,RANDOMFOREST,Decision_Tree,gmbScore, acc_gaussian, acc_sgd, acc_linear_svc]})
models.sort_values(by='Score', ascending=False)

In [ ]:
Y_prediction = RF.predict(X_test_Data)

In [ ]:
submission = pd.DataFrame({
        "PassengerId": test_Data["PassengerId"],
        "Survived": Y_prediction
    })

submission.to_csv('/kaggle/working/submission.csv', index=False)